In [39]:
!pip install geopandas shapely folium


In [40]:
import pandas as pd
import geopandas as gpd
import folium
from folium.plugins import MarkerCluster, HeatMap, Fullscreen, MousePosition
from shapely.geometry import Point


In [41]:
# Загрузка с правильным разделителем и пропущенной строкой заголовков
aero = pd.read_csv("data-62883-2025-06-16.csv", sep=';', skiprows=1)
bus = pd.read_csv("data-1881-2025-06-16.csv", sep=';', skiprows=1)
rail = pd.read_csv("data-62201-2025-06-16.csv", sep=';', skiprows=1)

# Очистка названий столбцов
for df in [aero, bus, rail]:
    df.columns = df.columns.str.strip()

# Унификация названий объектов
aero["name"] = aero["Наименование"]
bus["name"] = bus["Наименование автовокзала"]
rail["name"] = rail["Наименование"]

# Преобразование координат в точки
for df in [aero, bus, rail]:
    df["Широта в WGS-84"] = df["Широта в WGS-84"].astype(float)
    df["Долгота в WGS-84"] = df["Долгота в WGS-84"].astype(float)
    df["geometry"] = df.apply(lambda row: Point(row["Долгота в WGS-84"], row["Широта в WGS-84"]), axis=1)

# Преобразование в GeoDataFrame
aero_gdf = gpd.GeoDataFrame(aero, geometry="geometry", crs="EPSG:4326")
bus_gdf = gpd.GeoDataFrame(bus, geometry="geometry", crs="EPSG:4326")
rail_gdf = gpd.GeoDataFrame(rail, geometry="geometry", crs="EPSG:4326")


In [49]:
# Базовая карта
m = folium.Map(location=[55.75, 37.6], zoom_start=9, tiles="cartodb positron")

# ===== Слой 1: Аэропорты =====
aero_layer = folium.FeatureGroup(name="Аэропорты")
for _, row in aero_gdf.iterrows():
    popup = f"<b>{row['name']}</b><br>Аэроэкспресс: {row['Наличие Аэроэкспресса']}"
    color = "green" if str(row['Наличие Аэроэкспресса']).strip().lower() == "есть" else "red"
    folium.Marker(
        location=[row.geometry.y, row.geometry.x],
        tooltip=row["name"],
        popup=popup,
        icon=folium.Icon(color=color, icon="plane", prefix="fa")
    ).add_to(aero_layer)
m.add_child(aero_layer)

# ===== Слой 2: Автовокзалы (кластер) =====
bus_cluster = MarkerCluster(name="Автовокзалы")
for _, row in bus_gdf.iterrows():
    folium.Marker(
        location=[row.geometry.y, row.geometry.x],
        tooltip=row["name"],
        popup=row["name"],
        icon=folium.Icon(color="blue", icon="bus", prefix="fa")
    ).add_to(bus_cluster)
m.add_child(bus_cluster)

# ===== Слой 3: Ж/д вокзалы =====
rail_layer = folium.FeatureGroup(name="Ж/д вокзалы")
for _, row in rail_gdf.iterrows():
    folium.Marker(
        location=[row.geometry.y, row.geometry.x],
        tooltip=row["name"],
        popup=row["name"],
        icon=folium.Icon(color="orange", icon="train", prefix="fa")
    ).add_to(rail_layer)
m.add_child(rail_layer)

# ===== Слой 4: Тепловая карта =====
heat_data = []
for gdf in [aero_gdf, bus_gdf, rail_gdf]:
    heat_data += [[point.y, point.x] for point in gdf.geometry]
heat_layer = folium.FeatureGroup(name="Тепловая карта")
HeatMap(heat_data, radius=20).add_to(heat_layer)
m.add_child(heat_layer)



In [50]:
# Плагины
Fullscreen().add_to(m)
MousePosition().add_to(m)
folium.LayerControl(collapsed=False).add_to(m)

# Сохранение карты
m.save("index.html")